# Notebook probando predecir la probabilidad de éxito de cobro

In [115]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import parser

In [116]:
# Cargar los datos
df = pd.read_csv('../data/cobros_final.csv')

C:\Users\angel\AppData\Local\Temp\ipykernel_25320\1335326567.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/cobros_final.csv')


In [117]:
# Count nulls
nulls = df.isnull().sum()
nulls

idListaCobro                0
idCredito                   0
consecutivoCobro            0
idBanco                     0
montoExigible               0
montoCobrar                 0
montoCobrado                0
fechaCobroBanco       9679374
idRespuestaBanco       149561
fechaCreacionLista          0
fechaEnvioCobro             0
idEmisora                   0
dtype: int64

In [118]:
df.head(21)

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaCreacionLista,fechaEnvioCobro,idEmisora
0,100613,83021,27169966,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...
1,100613,87227,27169937,BANAMEX,517.74,517.74,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...
2,100613,103785,27169946,BANAMEX,681.23,681.23,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...
3,100613,110463,27169969,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...
4,100613,113755,27169948,BANAMEX,340.62,340.62,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...
5,100613,118237,27169957,BANAMEX,272.50,272.50,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...
6,100613,136004,27169958,BANAMEX,361.06,361.06,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...
7,100613,137382,27169934,BANAMEX,393.94,393.94,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...
8,100613,137885,27169970,BANAMEX,289.02,289.02,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...
9,100613,149399,27169968,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...


In [119]:
# Group by idCredito and sum the times montoCobrado is different than 0
countnopagar = df[df['montoCobrado'] == 0].groupby('idCredito').size().reset_index(name='non_zero_count')
countnopagar


,idCredito,non_zero_count
0,6,2
1,560,1
2,773,2
3,1154,2
4,1315,2
...,...,...
90186,755612,9
90187,755650,6
90188,755676,1
90189,755859,16


In [120]:
counttotal = df.groupby('idCredito')["idBanco"].count()
counttotal

idCredito
6          2
560        1
773        2
1154       2
1315       2
          ..
755683     1
755695     1
755726     1
755859    16
756595     5
Name: idBanco, Length: 97230, dtype: int64

In [121]:
# Select the index that appear on countnopagar that are not on counttotal
siemprepago = counttotal[~counttotal.index.isin(countnopagar['idCredito'])]
siemprepago

idCredito
20873     1
46038     2
46590     1
54690     1
57667     4
         ..
755668    1
755672    1
755683    1
755695    1
755726    1
Name: idBanco, Length: 7039, dtype: int64

In [122]:
# Remove the rows on countottal that are on siemprepago
counttotal = counttotal[~counttotal.index.isin(siemprepago.index)].reset_index()
counttotal

,idCredito,idBanco
0,6,2
1,560,1
2,773,2
3,1154,2
4,1315,2
...,...,...
90186,755612,9
90187,755650,6
90188,755676,1
90189,755859,16


In [123]:
# Divide counttotal['idBanco'] by countnopagar['idBanco']
counttotal["razon"] =  1 - countnopagar['non_zero_count'] / counttotal['idBanco']
counttotal

,idCredito,idBanco,razon
0,6,2,0.0
1,560,1,0.0
2,773,2,0.0
3,1154,2,0.0
4,1315,2,0.0
...,...,...,...
90186,755612,9,0.0
90187,755650,6,0.0
90188,755676,1,0.0
90189,755859,16,0.0


In [ ]:
# Add to df the column 'razon' with the values of counttotal['razon'] where df['idCredito'] == counttotal['idCredito']
#df = df.merge(counttotal[['idCredito', 'razon']], on='idCredito', how='left')
# Fill the nulls on 'razon' with 1
df

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaCreacionLista,fechaEnvioCobro,idEmisora,razon
0,100613,83021,27169966,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014320
1,100613,87227,27169937,BANAMEX,517.74,517.74,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000
2,100613,103785,27169946,BANAMEX,681.23,681.23,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000
3,100613,110463,27169969,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.006897
4,100613,113755,27169948,BANAMEX,340.62,340.62,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014134
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10057435,162271,741795,44299050,BANAMEX,980.32,980.32,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000
10057436,162271,741795,44299051,BANAMEX,1633.86,1633.86,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000
10057437,162271,742470,44299052,BANAMEX,810.44,810.44,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000
10057438,162271,742470,44299053,BANAMEX,1350.74,1350.74,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000


In [126]:
# Replace the nulls on 'razon' with 1
df['razon'] = df['razon'].fillna(1)

In [129]:
# Add a column to df 'pay' where 1 if montoCobrado > 0 else 0
df['pay'] = np.where(df['montoCobrado'] > 0, 1, 0)
df

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaCreacionLista,fechaEnvioCobro,idEmisora,razon,pay
0,100613,83021,27169966,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014320,0
1,100613,87227,27169937,BANAMEX,517.74,517.74,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
2,100613,103785,27169946,BANAMEX,681.23,681.23,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
3,100613,110463,27169969,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.006897,0
4,100613,113755,27169948,BANAMEX,340.62,340.62,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10057435,162271,741795,44299050,BANAMEX,980.32,980.32,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0
10057436,162271,741795,44299051,BANAMEX,1633.86,1633.86,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0
10057437,162271,742470,44299052,BANAMEX,810.44,810.44,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0
10057438,162271,742470,44299053,BANAMEX,1350.74,1350.74,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0


In [130]:
df.value_counts('pay')

pay
0    9679374
1     378066
Name: count, dtype: int64

In [131]:
df.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaCreacionLista,fechaEnvioCobro,idEmisora,razon,pay
0,100613,83021,27169966,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014320,0
1,100613,87227,27169937,BANAMEX,517.74,517.74,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
2,100613,103785,27169946,BANAMEX,681.23,681.23,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
3,100613,110463,27169969,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.006897,0
4,100613,113755,27169948,BANAMEX,340.62,340.62,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014134,0


In [146]:
# Generate a new dataframe with the columns 'idCredito', 'razon', 'pay' and 'fechaCobro'
df2 = df[['idBanco', 'montoExigible', 'fechaEnvioCobro','idEmisora','pay']].copy()
# Transform idBanco,idEmisora into one hot encoding
df2 = pd.get_dummies(df2, columns=['idBanco', 'idEmisora'], drop_first=True)
# Transform fechaEnvioCobro into month and day
df2['fechaEnvioCobro'] = pd.to_datetime(df2['fechaEnvioCobro'], format='%d/%m/%Y')
df2['month'] = df2['fechaEnvioCobro'].dt.month
df2['day'] = df2['fechaEnvioCobro'].dt.day
# Drop the column 'fechaEnvioCobro'
df2.drop(columns=['fechaEnvioCobro'], inplace=True)

In [158]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

# Features and target
X = df2.drop(columns=['pay'])
y = df2['pay']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Create and train the model
model = GaussianNB()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.89      0.94   2903812
           1       0.19      0.68      0.30    113420

    accuracy                           0.88   3017232
   macro avg       0.59      0.79      0.62   3017232
weighted avg       0.96      0.88      0.91   3017232



In [157]:
probs = model.predict_proba(X_test)[:, 1]
y_pred = (probs > 0.00001).astype(int)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.88      0.93   2903768
           1       0.18      0.71      0.29    113464

    accuracy                           0.87   3017232
   macro avg       0.59      0.79      0.61   3017232
weighted avg       0.96      0.87      0.91   3017232



In [152]:
probs

array([9.99998753e-01, 6.71190737e-09, 9.87309158e-07, ...,
       3.02887609e-16, 2.84316549e-06, 4.93121759e-09], shape=(3017232,))

In [ ]:
# Count 

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaCreacionLista,fechaEnvioCobro,idEmisora,razon,pay
0,100613,83021,27169966,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014320,0
1,100613,87227,27169937,BANAMEX,517.74,517.74,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
2,100613,103785,27169946,BANAMEX,681.23,681.23,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
3,100613,110463,27169969,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.006897,0
4,100613,113755,27169948,BANAMEX,340.62,340.62,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10057435,162271,741795,44299050,BANAMEX,980.32,980.32,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0
10057436,162271,741795,44299051,BANAMEX,1633.86,1633.86,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0
10057437,162271,742470,44299052,BANAMEX,810.44,810.44,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0
10057438,162271,742470,44299053,BANAMEX,1350.74,1350.74,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0


In [144]:
counttotal.columns

Index(['idCredito', 'idBanco', 'razon'], dtype='object')

In [145]:
counttotal.loc[counttotal['idBanco']==1].count()

idCredito    19728
idBanco      19728
razon        19728
dtype: int64